In [89]:
import pandas as pd
import numpy as np

df = pd.read_csv('dataset2.csv')

print(df.head(), df.shape)

   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520 (1337, 7)


In [90]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.preprocessing import PolynomialFeatures, FunctionTransformer
from feature_engine.discretisation import ArbitraryDiscretiser
from sklearn.model_selection import GridSearchCV
import math

# separation des features et de la variable cible
X = df.drop('charges', axis=1)
y = df[['charges']]
print(f'''verif des dimensions X et Y
      X (dataset sans la variable cible): {X.shape}
      Y (la variable cible) : {y.shape}''')


# division du dataset en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])
print(f''' verif du split 80 20 
80% du dataset : X train -> {X_train.shape}, Y train -> {y_train.shape}
20% du dataset : X test -> {X_test.shape}, Y test -> {y_test.shape}''')

# function custom
def log_transform(x):
    return np.log(x + 1)

def custom_bmi_discretizer(df):
    custom_bins = [0, 30, 100, float('inf')]
    bmi_discretizer = ArbitraryDiscretiser(binning_dict={'bmi': custom_bins}, return_object=True)
    df[['bmi']] = bmi_discretizer.fit_transform(df[['bmi']])
    return df

log_transformer = FunctionTransformer(log_transform)
bmi_discretizer_transformer = FunctionTransformer(custom_bmi_discretizer)

#### ======= preprocessing
#processeur pour split le bmi 
preprocessor_bmi = Pipeline(steps=[
    ('bmi_discretize', bmi_discretizer_transformer)
])

#processeur pour les colonnes numériques 
preprocessor_num = Pipeline(steps=[
    ('log', FunctionTransformer(log_transform)),
])
#processeur pour les colonnes catégorielles 
preprocessor_cat = Pipeline(steps=[
    ('encoder', OneHotEncoder())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('bmi', preprocessor_bmi, ['bmi']),
        ('num', preprocessor_num, ['age', 'bmi', 'children']),
        ('cat', preprocessor_cat, ['region', 'sex', 'smoker'])
    ]
)


verif des dimensions X et Y
      X (dataset sans la variable cible): (1337, 6)
      Y (la variable cible) : (1337, 1)
 verif du split 80 20 
80% du dataset : X train -> (1136, 6), Y train -> (1136, 1)
20% du dataset : X test -> (201, 6), Y test -> (201, 1)


In [91]:
from sklearn.dummy import DummyRegressor

# création du pipeline dummy
pipeline_dummy = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', DummyRegressor(strategy='mean'))])


# eviter data leakage -> entraîner le pipeline sur les données d'entraînement puis predire
# sur l'ensemble de test avec le meme pipeline
pipeline_dummy.fit(X_train, y_train)
#puis predire y sur l'ensemble de test avec le meme pipeline
y_pred = pipeline_dummy.predict(X_test)
#print(y_pred_dummy)

#test avec les differentes métriques pour le modele dummy : 
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error du modèle dummy: {mse}')

r2 = r2_score(y_test, y_pred)

print(f'Coefficient de determination R² du modèle dummy: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle dummy: {rmse}')

Mean Squared Error du modèle dummy: 145320521.96584958
Coefficient de determination R² du modèle dummy: -0.0027928862543942223
Root Mean Squared Error (RMSE) du modèle dummy: 12054.89618229247


In [92]:
# Calculez le RMSE (la racine carrée du MSE) pour obtenir une idée de l'erreur en termes des unités d'origine de vos données. 
# Comparez le RMSE à la moyenne des 'charges' (df['charges'].mean()) pour évaluer l'erreur relative.

# RMSE plus élevé que la moyenne : le modèle a du mal à capturer la variation et les tendances dans les données. 
# Il prédit généralement des valeurs qui sont relativement éloignées des valeurs réelles.

# Erreur systématique :  le modèle a tendance à sous-estimer ou surestimer systématiquement les valeurs cibles.

# Modèle insuffisamment performant : Un RMSE élevé peut également indiquer que le modèle que vous utilisez n'est pas suffisamment complexe 
# pour capturer les relations présentes dans les données. Cela peut signifier que le modèle ne tient pas compte de toutes les caractéristiques
# pertinentes ou que la stratégie de prédiction choisie (comme la moyenne) n'est pas adaptée aux données.

# En général, un bon modèle de régression devrait avoir un RMSE inférieur à la moyenne de la variable cible, 
# car cela indiquerait que le modèle est capable de faire des prédictions qui se rapprochent des valeurs réelles en moyenne. 
# L'objectif est de minimiser le RMSE pour obtenir des prédictions précises.

print(df['charges'].mean())
print(rmse)

#donc on en résume que le modele dummy est tres mauvais, et c'est normal car le modele dummy 
#en général, le modèle dummy est un modèle très simple et naïf qui est utilisé comme baseline 
#pour évaluer la performance d'autres modèles plus sophistiqués.

13279.121486655948
12054.89618229247


In [93]:
# Modele de regression lineaire

# création du pipeline de prétraitement et du modèle linear regression
pipeline_lr = Pipeline(steps=[('preprocessor', preprocessor),
                              ('poly', PolynomialFeatures(degree=2, include_bias=False)),
                            ('scaler', StandardScaler()),
                           ('regression', LinearRegression())])

pipeline_lr.fit(X_train, y_train)
y_pred = pipeline_lr.predict(X_test)
# print(y_pred)

In [94]:
#test des les differents metriques sur modele linear regression
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error du modèle linear regression: {mse}')

r2 = r2_score(y_test, y_pred)

print(f'Coefficient de determination R² du modèle linear regression: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle linear regression: {rmse}')

Mean Squared Error du modèle linear regression: 11250047.625469193
Coefficient de determination R² du modèle linear regression: 0.9223683786967487
Root Mean Squared Error (RMSE) du modèle linear regression: 3354.1090658279427


donc on en résume que le modele dummy est tres mauvais, et c'est normal car le modele dummy <br>
est un modèle très simple et naïf qui est utilisé comme baseline pour évaluer la performance d'autres modèles plus sophistiqués

In [95]:
# Modele de regression RIDGE

# création du pipeline de prétraitement et du modèle RIDGE
pipeline_ridge = Pipeline(steps=[('preprocessor', preprocessor),
                            ('poly', PolynomialFeatures(degree=2, include_bias=False)),
                            ('scaler', StandardScaler()),
                           ('regression', Ridge())])


#test de différents hyperparam d'alpha pour le lasso
param_grid_ridge = {
    'regression__alpha': np.arange(0.01,1,0.01)
}
#[0.01, 0.1, 1, 10, 100] 
### grid search
grid_ridge = GridSearchCV(pipeline_ridge, param_grid_ridge, cv=5)


# eviter data leakage -> entraîner le pipeline sur les données d'entraînement 
grid_ridge.fit(X_train, y_train)
best_model = grid_ridge.best_estimator_
#puis predire y sur l'ensemble de test avec le meme pipeline
y_pred = best_model.predict(X_test)
# print(y_pred)
#test des les differents metriques sur modele lasso
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)

r2 = r2_score(y_test, y_pred)

print(f'Coefficient de determination R² du modèle ridge: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle ridge: {rmse} \n rappel moyenne charge : 13279')



Coefficient de determination R² du modèle ridge: 0.9246401669241054
Root Mean Squared Error (RMSE) du modèle ridge: 3304.667856661573 
 rappel moyenne charge : 13279


In [96]:
best_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('bmi',
                                                  Pipeline(steps=[('bmi_discretize',
                                                                   FunctionTransformer(func=<function custom_bmi_discretizer at 0x0000021C2F9E72E0>))]),
                                                  ['bmi']),
                                                 ('num',
                                                  Pipeline(steps=[('log',
                                                                   FunctionTransformer(func=<function log_transform at 0x0000021C33DEF520>))]),
                                                  ['age', 'bmi', 'children']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  ['region', 'sex',
                                                   'smoker'])])),
                ('poly', PolynomialFeatures(include_bias=False)),
                ('scaler', StandardScaler()),
                ('regression', Ridge(alpha=0.12))])